In [ ]:
seeds = range(1, 11)

# Config should match Problem init

# BBOB

In [10]:
# BBOB
from omegaconf import DictConfig, OmegaConf
from pathlib import Path
import shutil

target_path = Path("smacbenchmarking/benchmarks/configs/bbob/problems/")
shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "smacbenchmarking.benchmarks.bbob.BBOBProblem"

fids = list(range(1, 25))
instances = [0]
dimensions = [4]

for fid in fids:
    for inst in instances:
        for dim in dimensions:
            cfg = DictConfig({
                "problem_class": {
                    "_target_": problem_class,
                    "dimension": dim,
                    "fid": fid,
                    "instance": inst,
                    "seed": "${seed}",  # get the seed from global cfg                      
                }
            })
            fn = target_path / f"{dim}_{fid}_{dim}_{inst}.yaml"
            yaml_str = OmegaConf.to_yaml(cfg=cfg)
            fn.write_text(yaml_str)
            print(cfg)



{'problem_class': {'_target_': 'smacbenchmarking.benchmarks.bbob.BBOBProblem', 'dimension': 4, 'fid': 1, 'instance': 0, 'seed': '${seed}'}}
{'problem_class': {'_target_': 'smacbenchmarking.benchmarks.bbob.BBOBProblem', 'dimension': 4, 'fid': 2, 'instance': 0, 'seed': '${seed}'}}
{'problem_class': {'_target_': 'smacbenchmarking.benchmarks.bbob.BBOBProblem', 'dimension': 4, 'fid': 3, 'instance': 0, 'seed': '${seed}'}}
{'problem_class': {'_target_': 'smacbenchmarking.benchmarks.bbob.BBOBProblem', 'dimension': 4, 'fid': 4, 'instance': 0, 'seed': '${seed}'}}
{'problem_class': {'_target_': 'smacbenchmarking.benchmarks.bbob.BBOBProblem', 'dimension': 4, 'fid': 5, 'instance': 0, 'seed': '${seed}'}}
{'problem_class': {'_target_': 'smacbenchmarking.benchmarks.bbob.BBOBProblem', 'dimension': 4, 'fid': 6, 'instance': 0, 'seed': '${seed}'}}
{'problem_class': {'_target_': 'smacbenchmarking.benchmarks.bbob.BBOBProblem', 'dimension': 4, 'fid': 7, 'instance': 0, 'seed': '${seed}'}}
{'problem_class': {'